In [1]:
import gymnasium as gym
import wandb
from typing import Callable
from stable_baselines3 import PPO
from Educational.args import args
from Educational.SFQenv import SFQ
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import HParam
from wandb.integration.sb3 import WandbCallback
from pprint import pprint

ModuleNotFoundError: No module named 'Educational'

In [2]:
wandb.login()

wandb: Currently logged in as: donalexs12. Use `wandb login --relogin` to force relogin


True

In [3]:
sweep_config = {
    'method': 'random',
    'early_terminate':{
        'type':'hyperband',
        'eta':2,
        'min_iter':90,
        's':3
    }
    }

In [4]:
metric = {
    'name': 'rollout/ep_rew_mean',
    'goal': 'maximize',
    'target':40
    }

sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'ent_coef': {
        'values': [0.00001, 0.0001, 0.001, 0.01]
    },
    # 'policy_layer_size': {
    #     'values': [64, 128, 256]
    # },
    # 'value_layer_size': {
    #     'values': [64, 128, 256]
    # },
    'vf_coef':{
        'values':[0.25,0.5,0.75]
    },
    'n_epochs':{
        'values':[5,10,15]
    }
}

In [6]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },
    'batch_size': {
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 100,
        'max': 6000
    }
})

In [7]:
sweep_config['parameters']=parameters_dict

In [8]:
pprint(sweep_config)

{'early_terminate': {'eta': 2, 'min_iter': 90, 's': 3, 'type': 'hyperband'},
 'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'train/loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 6000,
                               'min': 100,
                               'q': 8},
                'ent_coef': {'values': [1e-05, 0.0001, 0.001, 0.01]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'n_epochs': {'values': [5, 10, 15]},
                'vf_coef': {'values': [0.25, 0.5, 0.75]}}}


In [9]:
sweep_id = wandb.sweep(sweep_config, project="SFQsweep")

Create sweep with ID: b8wy3uaa
Sweep URL: https://wandb.ai/donalexs12/SFQsweep/sweeps/b8wy3uaa


In [10]:
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """

    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [11]:
def train_PPO(config=None):
    with wandb.init(config=config,
                    sync_tensorboard=True,
                    project='SFQ_hyperparam'):
        config=wandb.config
        env = SFQ()
        vec_env = make_vec_env(SFQ, n_envs=14)
        # policy_layer_size = config.policy_layer_size
        # value_layer_size = config.value_layer_size
        # policy = ActorCriticPolicy(net_arch=dict(pi=[policy_layer_size,policy_layer_size], vf = [value_layer_size,value_layer_size]))
        model = PPO("MlpPolicy",vec_env,
                    verbose=0,
                    batch_size=config.batch_size,
                    ent_coef=config.ent_coef,
                    learning_rate=linear_schedule(config.learning_rate),
                    tensorboard_log=f"new_PPO_runs/")
        model.learn(
            total_timesteps=int(10e6),
            callback=WandbCallback(
                                   gradient_save_freq=100,
                                   model_save_path=f"models/",)
        )

In [ ]:
wandb.agent(sweep_id, train_PPO, count=100)

wandb: Agent Starting Run: 3axjm25r with config:
wandb: 	batch_size: 2424
wandb: 	ent_coef: 0.001
wandb: 	learning_rate: 0.09126489294346882
wandb: 	n_epochs: 5
wandb: 	vf_coef: 0.25
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\user\PycharmProjects\Educational\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:154: UserWarning: You have specified a mini-batch size of 2424, but because the `RolloutBuffer` is of size `n_steps * n_envs = 28672`, after every 11 untruncated mini-batches, there will be a truncated mini-batch of size 2008
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2048 and n_envs=14)
  warnings.warn(
C:\Users\user\PycharmProjects\Educational\Educational\SFQ_calc.py:99: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(complex128, 2, 'C', True, aligned=True), Array(complex128, 2, 'A', True, aligned=True))
  ket = u_matrix @ alpha_state


global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
rollout/ep_len_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
rollout/ep_rew_mean,▁█▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
time/fps,▂▁█▇▅▃▄▄▅▅▃▄▄▄▄▅▅▅▅▅▆▆▅▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆
train/approx_kl,▃█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/clip_fraction,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/clip_range,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/entropy_loss,▁▆██████████████████████████████████████
train/explained_variance,▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▁▅█▇▆▅▄▅▅▅▇▆▅▇▆▆▆▆▅▆▆▆▅▆▅▆▇▆▆▇▆▆▆▇▅▆▆▅▆▅


wandb: Agent Starting Run: yl90gel6 with config:
wandb: 	batch_size: 560
wandb: 	ent_coef: 0.001
wandb: 	learning_rate: 0.07973987421998159
wandb: 	n_epochs: 5
wandb: 	vf_coef: 0.25
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\user\PycharmProjects\Educational\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:154: UserWarning: You have specified a mini-batch size of 560, but because the `RolloutBuffer` is of size `n_steps * n_envs = 28672`, after every 51 untruncated mini-batches, there will be a truncated mini-batch of size 112
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2048 and n_envs=14)
  warnings.warn(
wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


...

wandb: Agent Starting Run: sssl2qv4 with config:
wandb: 	batch_size: 216
wandb: 	ent_coef: 0.0001
wandb: 	learning_rate: 0.09720920383134918
wandb: 	n_epochs: 15
wandb: 	vf_coef: 0.25
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\user\PycharmProjects\Educational\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:154: UserWarning: You have specified a mini-batch size of 216, but because the `RolloutBuffer` is of size `n_steps * n_envs = 28672`, after every 132 untruncated mini-batches, there will be a truncated mini-batch of size 160
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2048 and n_envs=14)
  warnings.warn(
wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
rollout/ep_len_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
rollout/ep_rew_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time/fps,█▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/approx_kl,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/clip_fraction,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/clip_range,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/entropy_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/explained_variance,▅▁█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅█▅▅▆█▅▅▅▅▅▅█▅▅█▅▅▁▁▅█
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▂▅▄▂▄▇▄▃▄▄▃▅▆▅▆▄▅▅▅▄▄▅▂▃▂▅▃▄▃▅▃▆▂▂▁▁▃▄


wandb: Agent Starting Run: lr41lvq5 with config:
wandb: 	batch_size: 120
wandb: 	ent_coef: 0.01
wandb: 	learning_rate: 0.008940745028571761
wandb: 	n_epochs: 5
wandb: 	vf_coef: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\user\PycharmProjects\Educational\venv\lib\site-packages\stable_baselines3\ppo\ppo.py:154: UserWarning: You have specified a mini-batch size of 120, but because the `RolloutBuffer` is of size `n_steps * n_envs = 28672`, after every 238 untruncated mini-batches, there will be a truncated mini-batch of size 112
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2048 and n_envs=14)
  warnings.warn(
wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


In [ ]:
policy = ActorCriticPolicy(net_arch=dict(pi=[run_config.policy_layer_size, run_config.policy_layer_size],
     vf=[run_config.value_layer_size, run_config.value_layer_size]))